In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

Data Collection and Pre-Processing


In [ ]:
#loading the data from the csv file to a panda dataframe

movies_data = pd.read_csv('/content/movies.csv')



In [ ]:
#printing first 5 rows of dataframe
movies_data.head()


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
#number of rows and cols

movies_data.shape

(4803, 24)

In [ ]:
#selecting relevent features for recommendation

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [ ]:
# replacing the null valuess with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [ ]:
movies_data

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director,combined_features
0,0,237000000,<bound method Series.fillna of 0 Action ...,http://www.avatarmovie.com/,19995,<bound method Series.fillna of 0 culture...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,<bound method Series.fillna of 0 ...,Avatar,7.2,11800,<bound method Series.fillna of 0 Sam Wor...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",<bound method Series.fillna of 0 Jam...,<bound method Series.fillna of 0 Action ...
1,1,300000000,<bound method Series.fillna of 0 Action ...,http://disney.go.com/disneypictures/pirates/,285,<bound method Series.fillna of 0 culture...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,<bound method Series.fillna of 0 ...,Pirates of the Caribbean: At World's End,6.9,4500,<bound method Series.fillna of 0 Sam Wor...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",<bound method Series.fillna of 0 Jam...,<bound method Series.fillna of 0 Action ...
2,2,245000000,<bound method Series.fillna of 0 Action ...,http://www.sonypictures.com/movies/spectre/,206647,<bound method Series.fillna of 0 culture...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,<bound method Series.fillna of 0 ...,Spectre,6.3,4466,<bound method Series.fillna of 0 Sam Wor...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",<bound method Series.fillna of 0 Jam...,<bound method Series.fillna of 0 Action ...
3,3,250000000,<bound method Series.fillna of 0 Action ...,http://www.thedarkknightrises.com/,49026,<bound method Series.fillna of 0 culture...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,<bound method Series.fillna of 0 ...,The Dark Knight Rises,7.6,9106,<bound method Series.fillna of 0 Sam Wor...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",<bound method Series.fillna of 0 Jam...,<bound method Series.fillna of 0 Action ...
4,4,260000000,<bound method Series.fillna of 0 Action ...,http://movies.disney.com/john-carter,49529,<bound method Series.fillna of 0 culture...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,<bound method Series.fillna of 0 ...,John Carter,6.1,2124,<bound method Series.fillna of 0 Sam Wor...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",<bound method Series.fillna of 0 Jam...,<bound method Series.fillna of 0 Action ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,<bound method Series.fillna of 0 Action ...,NaN,9367,<bound method Series.fillna of 0 culture...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,<bound method Series.fillna of 0 ...,El Mariachi,6.6,238,<bound method Series.fillna of 0 Sam Wor...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",<bound method Series.fillna of 0 Jam...,<bound method Series.fillna of 0 Action ...
4799,4799,9000,<bound method Series.fillna of 0 Action ...,NaN,72766,<bound method Series.fillna of 0 culture...,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,[],Released,<bound method Series.fillna of 0 ...,Newlyweds,5.9,5,<bound method Series.fillna of 0 Sam Wor...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",<bound method Series.fillna of 0 Jam...,<bound method Series.fillna of 0 Action ...
4800,4800,0,<bound method Series.fillna of 0 Action ...,http://www.hallmarkchannel

In [ ]:
# Combine features into a single column
movies_data['combined_features'] = movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['tagline'] + ' ' + movies_data['cast'] + ' ' + movies_data['director']

# Display the first few rows to inspect the combined features
print(movies_data['combined_features'].head())


0    <bound method Series.fillna of 0       Action ...
1    <bound method Series.fillna of 0       Action ...
2    <bound method Series.fillna of 0       Action ...
3    <bound method Series.fillna of 0       Action ...
4    <bound method Series.fillna of 0       Action ...
Name: combined_features, dtype: object


In [ ]:
print(combined_features)

0       <bound method Series.fillna of 0       Action ...
1       <bound method Series.fillna of 0       Action ...
2       <bound method Series.fillna of 0       Action ...
3       <bound method Series.fillna of 0       Action ...
4       <bound method Series.fillna of 0       Action ...
                              ...                        
4798    <bound method Series.fillna of 0       Action ...
4799    <bound method Series.fillna of 0       Action ...
4800    <bound method Series.fillna of 0       Action ...
4801    <bound method Series.fillna of 0       Action ...
4802    <bound method Series.fillna of 0       Action ...
Length: 4803, dtype: object


In [ ]:
#converting the text data to feature vector

vectorizer = TfidfVectorizer()


In [ ]:
# Fit and transform the combined features into feature vectors
feature_vectors = vectorizer.fit_transform(movies_data['combined_features'])


In [ ]:
# Display the shape of the feature vectors to ensure they are correctly generated
print(feature_vectors.shape)

(4803, 201)


In [ ]:
print(feature_vectors)

  (0, 59)	0.035578403348241
  (0, 95)	0.035578403348241
  (0, 169)	0.035578403348241
  (0, 161)	0.035578403348241
  (0, 154)	0.035578403348241
  (0, 153)	0.035578403348241
  (0, 173)	0.035578403348241
  (0, 11)	0.035578403348241
  (0, 136)	0.035578403348241
  (0, 37)	0.035578403348241
  (0, 124)	0.035578403348241
  (0, 192)	0.035578403348241
  (0, 89)	0.035578403348241
  (0, 32)	0.035578403348241
  (0, 103)	0.035578403348241
  (0, 34)	0.035578403348241
  (0, 76)	0.035578403348241
  (0, 44)	0.035578403348241
  (0, 92)	0.071156806696482
  (0, 26)	0.071156806696482
  (0, 18)	0.035578403348241
  (0, 63)	0.035578403348241
  (0, 156)	0.035578403348241
  (0, 10)	0.035578403348241
  (0, 149)	0.035578403348241
  :	:
  (4802, 130)	0.177892016741205
  (4802, 60)	0.035578403348241
  (4802, 4)	0.177892016741205
  (4802, 131)	0.177892016741205
  (4802, 3)	0.177892016741205
  (4802, 129)	0.035578403348241
  (4802, 186)	0.035578403348241
  (4802, 2)	0.177892016741205
  (4802, 155)	0.071156806696482
  

Cosine Similarity


In [ ]:
# Compute cosine similarity matrix

similarity = cosine_similarity(feature_vectors)

In [ ]:
# Display the similarity matrix to inspect the values

print(similarity)

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [ ]:
print(similarity.shape)


(4803, 4803)


Getting Movie Name from user


In [ ]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : iron man


In [ ]:
#creating a list with all movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [ ]:
close_match = find_close_match[0]
print(close_match)

Iron Man


In [ ]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

68


In [ ]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.9999999999999998), (1, 0.9999999999999998), (2, 0.9999999999999998), (3, 0.9999999999999998), (4, 0.9999999999999998), (5, 0.9999999999999998), (6, 0.9999999999999998), (7, 0.9999999999999998), (8, 0.9999999999999998), (9, 0.9999999999999998), (10, 0.9999999999999998), (11, 0.9999999999999998), (12, 0.9999999999999998), (13, 0.9999999999999998), (14, 0.9999999999999998), (15, 0.9999999999999998), (16, 0.9999999999999998), (17, 0.9999999999999998), (18, 0.9999999999999998), (19, 0.9999999999999998), (20, 0.9999999999999998), (21, 0.9999999999999998), (22, 0.9999999999999998), (23, 0.9999999999999998), (24, 0.9999999999999998), (25, 0.9999999999999998), (26, 0.9999999999999998), (27, 0.9999999999999998), (28, 0.9999999999999998), (29, 0.9999999999999998), (30, 0.9999999999999998), (31, 0.9999999999999998), (32, 0.9999999999999998), (33, 0.9999999999999998), (34, 0.9999999999999998), (35, 0.9999999999999998), (36, 0.9999999999999998), (37, 0.9999999999999998), (38, 0.99999999999999

In [ ]:
len(similarity_score)

4803

In [ ]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(0, 0.9999999999999998), (1, 0.9999999999999998), (2, 0.9999999999999998), (3, 0.9999999999999998), (4, 0.9999999999999998), (5, 0.9999999999999998), (6, 0.9999999999999998), (7, 0.9999999999999998), (8, 0.9999999999999998), (9, 0.9999999999999998), (10, 0.9999999999999998), (11, 0.9999999999999998), (12, 0.9999999999999998), (13, 0.9999999999999998), (14, 0.9999999999999998), (15, 0.9999999999999998), (16, 0.9999999999999998), (17, 0.9999999999999998), (18, 0.9999999999999998), (19, 0.9999999999999998), (20, 0.9999999999999998), (21, 0.9999999999999998), (22, 0.9999999999999998), (23, 0.9999999999999998), (24, 0.9999999999999998), (25, 0.9999999999999998), (26, 0.9999999999999998), (27, 0.9999999999999998), (28, 0.9999999999999998), (29, 0.9999999999999998), (30, 0.9999999999999998), (31, 0.9999999999999998), (32, 0.9999999999999998), (33, 0.9999999999999998), (34, 0.9999999999999998), (35, 0.9999999999999998), (36, 0.9999999999999998), (37, 0.9999999999999998), (38, 0.99999999999999

In [ ]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Avatar
2 . Pirates of the Caribbean: At World's End
3 . Spectre
4 . The Dark Knight Rises
5 . John Carter
6 . Spider-Man 3
7 . Tangled
8 . Avengers: Age of Ultron
9 . Harry Potter and the Half-Blood Prince
10 . Batman v Superman: Dawn of Justice
11 . Superman Returns
12 . Quantum of Solace
13 . Pirates of the Caribbean: Dead Man's Chest
14 . The Lone Ranger
15 . Man of Steel
16 . The Chronicles of Narnia: Prince Caspian
17 . The Avengers
18 . Pirates of the Caribbean: On Stranger Tides
19 . Men in Black 3
20 . The Hobbit: The Battle of the Five Armies
21 . The Amazing Spider-Man
22 . Robin Hood
23 . The Hobbit: The Desolation of Smaug
24 . The Golden Compass
25 . King Kong
26 . Titanic
27 . Captain America: Civil War
28 . Battleship
29 . Jurassic World


Movie Recommendation System


In [ ]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : BatMan
Movies suggested for you : 

1 . Avatar
2 . Pirates of the Caribbean: At World's End
3 . Spectre
4 . The Dark Knight Rises
5 . John Carter
6 . Spider-Man 3
7 . Tangled
8 . Avengers: Age of Ultron
9 . Harry Potter and the Half-Blood Prince
10 . Batman v Superman: Dawn of Justice
11 . Superman Returns
12 . Quantum of Solace
13 . Pirates of the Caribbean: Dead Man's Chest
14 . The Lone Ranger
15 . Man of Steel
16 . The Chronicles of Narnia: Prince Caspian
17 . The Avengers
18 . Pirates of the Caribbean: On Stranger Tides
19 . Men in Black 3
20 . The Hobbit: The Battle of the Five Armies
21 . The Amazing Spider-Man
22 . Robin Hood
23 . The Hobbit: The Desolation of Smaug
24 . The Golden Compass
25 . King Kong
26 . Titanic
27 . Captain America: Civil War
28 . Battleship
29 . Jurassic World
